In [3]:
from pdf_handler import PDFHandler

import fitz
import pandas as pd
import numpy as np
import os
import sqlite3


In [4]:
df = pd.read_csv("regex_validation.csv")
df

,pdf_name,section_type
0,amraoui2022_splc,numeric_section
1,cheng2023_tse,numeric_section
2,dorn2020_vamos,numeric_section
3,friesel2022_icse,numeric_section
4,gong2022_msr,numeric_section
...,...,...
116,tipu2022_cc,numeric_section
117,vázquez-ingelmo2020_cc,numeric_section
118,velez2020ase,numeric_section
119,Vitui2021ese,numeric_section


In [5]:
def find_all_paths_and_names(folder_path):
    all_paths = []
    all_names = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            all_paths.append(os.path.join(root, file))
            all_names.append(file)
    return all_paths, all_names

# Example usage
all_file_paths, all_file_names = find_all_paths_and_names("/home/pedro/Documents/Rag_test/grpc/papers_pdf")

In [6]:

#path = "/home/pedro/Documents/Rag_test/grpc/papers_pdf/ScienceDirect/Arcaini2020.pdf"
    
def extract_titles_sections(path, section_pattern):
    doc = PDFHandler.try_open(path)
    conn = sqlite3.connect("extracted_titles.db")
    cursor = conn.cursor()
    
    # Create tables if they don't exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS pdfs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            pdf_name TEXT NOT NULL UNIQUE,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS extracted_text (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            pdf_id INTEGER NOT NULL,
            sequence INTEGER NOT NULL,
            title TEXT,
            text TEXT,
            FOREIGN KEY (pdf_id) REFERENCES pdfs (id)
        )
    """)
    
    # Check if PDF already exists
    pdf_name = os.path.basename(path)
    cursor.execute("SELECT id FROM pdfs WHERE pdf_name = ?", (pdf_name,))
    existing_pdf = cursor.fetchone()
    
    if existing_pdf:
        pdf_id = existing_pdf[0]
        # Optional: Delete existing text entries for this PDF to update them
        cursor.execute("DELETE FROM extracted_text WHERE pdf_id = ?", (pdf_id,))
    else:
        # Insert new PDF record
        cursor.execute("INSERT INTO pdfs (pdf_name) VALUES (?)", (pdf_name,))
        pdf_id = cursor.lastrowid
    
    # Extract text
    taged_text, _, smode = PDFHandler.tagged_extraction(doc, "bold")
    regex_pattern = PDFHandler.regex_patterns[section_pattern]
    extracted_text = PDFHandler.extract_text_with_regex(taged_text, regex_pattern, smode)
    
    # Insert extracted text into the table with sequence
    sequence = 1
    for key, value in extracted_text.items():
        cursor.execute("""
            INSERT INTO extracted_text (pdf_id, sequence, title, text) 
            VALUES (?, ?, ?, ?)
        """, (pdf_id, sequence, value["title"], value["text"]))
        sequence += 1
    
    # Commit changes and close the connection
    conn.commit()
    conn.close()
    doc.close()
    
    return pdf_id

In [11]:
# Change the path to your folder with PDF files
all_file_paths, all_file_names = find_all_paths_and_names("/home/PUC/Documentos/AutoSLR/papers_pdf")

In [12]:
for idx, file in enumerate(all_file_paths):
    try:
        section = df.loc[df["pdf_name"] == all_file_names[idx][:-4], "section_type"].values[0]
        if section == "null":
            continue
        extract_titles_sections(file, section)
    except Exception as e:
        print(f"Error processing {file}: {e}")
        continue

Error processing /home/PUC/Documentos/AutoSLR/papers_pdf/Scopus/tërnava2022.pdf: index 0 is out of bounds for axis 0 with size 0
Error processing /home/PUC/Documentos/AutoSLR/papers_pdf/Scopus/mehlstäubl2022.pdf: index 0 is out of bounds for axis 0 with size 0
Error processing /home/PUC/Documentos/AutoSLR/papers_pdf/Scopus/Ha-2019ICSME.pdf: index 0 is out of bounds for axis 0 with size 0
Error processing /home/PUC/Documentos/AutoSLR/papers_pdf/Scopus/Ha2019-icse.pdf: index 0 is out of bounds for axis 0 with size 0
Error processing /home/PUC/Documentos/AutoSLR/papers_pdf/Scopus/Yu2021.pdf: index 0 is out of bounds for axis 0 with size 0
Error processing /home/PUC/Documentos/AutoSLR/papers_pdf/Scopus/Gao2021-ICSE.pdf: index 0 is out of bounds for axis 0 with size 0
Error processing /home/PUC/Documentos/AutoSLR/papers_pdf/SpringerLink/sewal2024cc.pdf: index 0 is out of bounds for axis 0 with size 0
Error processing /home/PUC/Documentos/AutoSLR/papers_pdf/ACM/gonzalez-rojas2019_otm.pdf: 